### Data Description

Data was gathered from the city of Chicago's public dataset: [2001 to present - Dashboard](https://data.cityofchicago.org/Public-Safety/Crimes-2001-to-present-Dashboard/5cd6-ry5g). API documentation can be found at [Socrata](https://dev.socrata.com/foundry/data.cityofchicago.org/ijzp-q8t2). Data was then loaded into a local MySQL database for analytics purposes. 

The data used for this analysis was pulled on 2023-08-01, note that data retrieved is up to the most recent day minus 7 days. For this reason, we must exclude the most recent month, year or day which does not have complete data for that time period if we want accurate results.

### Columns Available
* `id` = unique identified
* `case_number` = Chicago Police Department RD Number
* `date` = timestamp when incident occurred
* `block` = The partially redacted address where the incident occurred, placing it on the same block as the actual address
* `iucr` = The Illinois Uniform Crime Reporting code
* `primary_type` = The primary description of the IUCR code
* `description` = The secondary description of the IUCR code, a subcategory of the primary description
* `location_description` = Description of the location where the incident occurred
* `arrest` = Indicates whether an arrest was made
* `domestic` = Indicates whether the incident was domestic-related as defined by the Illinois Domestic Violence Act
* `beat` = Indicates the beat where the incident occurred
* `district` = Indicates the police district where the incident occurred
* `ward` = The ward (City Council district) where the incident occurred
* `community_area` = Indicates the community area where the incident occurred
* `fbi_code` = Indicates the crime classification as outlined in the FBI's National Incident-Based Reporting System (NIBRS)
* `x_coordinate` = The x coordinate of the location where the incident occurred in State Plane Illinois East NAD 1983 projection
* `y_coordinate` = The y coordinate of the location where the incident occurred in State Plane Illinois East NAD 1983 projection
* `year` = Year the incident occurred
* `updated_on` = Date and time the record was last updated
* `latitude` = The latitude of the location where the incident occurred
* `longitude` = The longitude of the location where the incident occurred
* `location` (not retrieved) = The location where the incident occurred in a format that allows for creation of maps and other geographic operations on this data portal
* `month` (custom) = The month when the incident occurred. Derived from `date`.
* `ymd` (custom) = The year, month and day when the incident occurred. Derived from `date` 

### Data Aggregation

Data was aggregated into a univariate datasets for the count of incidents by year, month and day. 

In [1]:
import pandas as pd

In [7]:
# crime counts by year
cc_year = pd.read_csv('data/cc_yearly_20230801.csv')
cc_year.head()

,year,yearly_crimes
0,2001,485878
1,2002,486799
2,2003,475984
3,2004,469419
4,2005,453769


In [3]:
# crime counts by month
cc_month = pd.read_csv('data/cc_monthly_20230801.csv')
cc_month.head()

,year,month,monthly_crimes
0,2001,1,38119
1,2001,2,33787
2,2001,3,40566
3,2001,4,40091
4,2001,5,41839


In [9]:
# crime counts by day
cc_day = pd.read_csv('data/cc_daily_20230801.csv')
cc_day.head()

,ymd,daily_crimes
0,2001-01-01,1825
1,2001-01-02,1143
2,2001-01-03,1151
3,2001-01-04,1166
4,2001-01-05,1267


### Time Plots

### Exporting "Cleaned Data"

As described previously, the data retrieved is up to the most recent day minus 7 days. For this reason, we must exclude the most recent day since it does not have complete data for that time period if we want accurate results.

In [10]:
def exclude_last_row_and_export(dataframe, output_file_path):
    # Use .iloc to slice the DataFrame and exclude the last row
    dataframe_excluded_last_row = dataframe.iloc[:-1]
    
    # Export the modified DataFrame to a CSV file
    dataframe_excluded_last_row.to_csv(output_file_path, index=False)

In [11]:
exclude_last_row_and_export(cc_day, 'data/cc_day_final.csv')